In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# 1 - Web scrapping
Nous allons récupérer les données sur le site de la [FIDE](https://ratings.fide.com/advseek.phtml) (Fédération Internationale Des Echecs), une organisation qui gère les fédérations nationales ainsi que les compétitions mondiales. 

Nous allons nous intéresser uniquement aux joueurs titrés : WFM et + ou tous les joueurs titrés

Etant donnné que les liens pour le scrapping ne sont pas fonctionnels (ou pas à tester)

In [2]:
def find_titled_years():
    year = 1908
    titled_year = {}
    for i in range(year,2018):
        url = 'https://ratings.fide.com/advaction.phtml?idcode=&name=&title=all_g&other_title=&country=%25&sex=&srating=0&erating=3000&birthday='+str(i)+'&radio=name&line=asc'
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        num = (soup.find('b').text)
        if num != '0':
            titled_year[i] = num
    return titled_year

In [3]:
%%time
#takes about 30sec to run
titled_year = find_titled_years()

Wall time: 35.3 s


Nous avons des joueurs titrés pour des personnes qui sont nées de 1920 à 2011, sauf l'an 1924

In [4]:
titled_year;

Fonction pour récupere toutes les données des joueurs

In [9]:
%%time
def scrap_players():
    url_1 = 'https://ratings.fide.com/advaction.phtml?idcode=&name=&title=all_g&other_title=&country=&sex=&srating=0&erating=3000&birthday='
    url_3 = '&radio=name&ex_rated=&line=asc&inactiv=&offset=' 
    titled_year = find_titled_years()
    merge = []
    for year in titled_year.keys():
        maximum = int(titled_year[year])
        #getting the url
        
        for amount in range(0,maximum,100):
            #Creation de l'url
            url = url_1 + str(year) + url_3 + str(amount)
            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, 'html.parser')
            #Trouve les bonnes balises pour extraire la data
            soup_tr = soup.find_all('tr', {'bgcolor':re.compile(r'#*')})
            for td in soup_tr:
                merge = merge + td.find_all('td')
    for i in range(len(merge)):
        merge[i] = re.sub(r'(\xa0)+',' ', merge[i].text)

    array = np.reshape(merge,(-1,12))
    return  pd.DataFrame(data = array,
                         columns=['IDcode','Name','T','WT','Oth. T.', 'Fed', 'Rtg', 'Rpd', 'Blz' ,'B-Year', 'S' ,'F'])
df = scrap_players()


Wall time: 2min 59s


Voila! Enregistions le dans un csv puisque le code prend assez de temps à récupérer les données

In [10]:
df

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F
0,5813298,"Lim, Kok Ann",CM,,,SGP,,,,1920,M,
1,4611870,"Keller-Hermann, Edith",WGM,WGM,,GER,2290,,,1921,F,i
2,2002310,"Levy, Louis",FM,,,USA,2275,,,1921,M,i
3,14100657,"Ousatchi, Mark",FM,,,GER,2157,,,1921,M,i
4,4101529,"Averbakh, Yuri L",GM,,,RUS,2445,,,1922,M,i
...,...,...,...,...,...,...,...,...,...,...,...,...
19505,24250775,"Zverev, Lev",CM,,,RUS,2163,2104,2186,2010,M,
19506,13429990,"Bashirli, Saadat",WCM,WCM,,AZE,1284,1344,1153,2011,F,
19507,44599790,"Erdogmus, Yagiz Kaan",CM,,,TUR,2150,1442,1783,2011,M,
19508,169013,"Schnaider, Ilan",CM,,,ARG,2154,1723,1800,2011,M,


In [11]:
df.to_csv('titled_players.csv')

In [12]:
df = pd.read_csv('titled_players.csv', index_col = 0)

Voyons voir si la Russie ou l'URSS est le pays qui possède le plus de joueurs titrés

In [13]:
df['Fed'].value_counts(normalize = True) * 100

RUS    13.316248
GER     7.524346
USA     4.146591
ESP     3.552025
SRB     3.018965
         ...    
SEN     0.005126
TAN     0.005126
TLS     0.005126
CAM     0.005126
BDI     0.005126
Name: Fed, Length: 180, dtype: float64

En effet c'est bien le cas